# UCB Extension Data Science Homework 4
Pymoli Exercise
written by: A. Lam

# Analysis:
- Observation 1: Player Demographics are primarily Male, between ages 20 and 30.
- Observation 2: Despite having upwards of 570 players, the most frequently purchased item has only been bought 11 times, meaning in-game purchase items are quite low.
- Observation 3: Without knowing what the mechanics of the game are or what these items are for, out of the most profitable items available for sale, only one of them is in the top ten most purchased items. 

In [1]:
# Modules
import os
import pandas as pd
import numpy as np

In [2]:
# Create path to File
# file 1
file_path = os.path.join('..','Instructions','HeroesOfPymoli','purchase_data.json')
# file 2
# file_path = os.path.join('..','Instructions','HeroesOfPymoli','purchase_data2.json')

In [3]:
# Read json file
json_data = pd.read_json(file_path)
json_data.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


# Player Count

In [4]:
player_count = pd.DataFrame({'Total Players' : [json_data['SN'].nunique()]})
player_count

,Total Players
0,573


Purchasing Analysis (total)

In [5]:
tot_purchase = pd.DataFrame({
    'Number of Unique Items' : [json_data['Item Name'].nunique()],
    'Average Purchase Price' : ['$' + str(round(json_data['Price'].mean(),2))],
    'Total Number of Purchases' : [len(json_data.index)],
    'Total Revenue' : ['$' + str(json_data['Price'].sum())]
    })
tot_purchase

,Average Purchase Price,Number of Unique Items,Total Number of Purchases,Total Revenue
0,$2.93,179,780,$2286.33


# Gender Demographics

In [6]:
# Create gender group
gender_data = json_data.groupby(['Gender'])
# Percentages calculated using player count value
demographics = pd.DataFrame(gender_data['SN'].nunique())
demographics = demographics.rename(columns = {'SN': 'Total Count'})
demographics['Percentage of Players'] = round(100*(gender_data['SN'].nunique()/player_count['Total Players'][0]),2)
demographics

,Total Count,Percentage of Players
Gender,,
Female,100,17.45
Male,465,81.15
Other / Non-Disclosed,8,1.40


# Purchasing Analysis (Gender)

In [7]:
purchase_by_gender = pd.concat([gender_data['SN'].count(),
    round(gender_data['Price'].mean(),2), 
    round(gender_data['Price'].sum(),2),
    round(gender_data['Price'].sum()/gender_data['SN'].nunique(),2)], axis = 1)
purchase_by_gender.columns = ['Purchase Count','Average Price','Total Value','Normalized Total']
# purchase_by_gender

# Age Demographics

In [8]:
# Setting up bins and group names
bins = [x * 5 for x in [0,2,3,4,6,8,10,12,16,20]]
group_names = []
for i in range(1,len(bins)):
    if i == 1:
        group_names.append('< ' + str(bins[i]))
    elif i == (len(bins)-1):
        group_names.append(str(bins[i-1]) + '+')    
    else:
        group_names.append(str(bins[i-1]) + ' - ' + str(bins[i]))

In [9]:
# bin data and create group
json_data_binned = json_data
json_data_binned['Age Group'] = pd.cut(json_data['Age'],bins,labels = group_names)
age_data = json_data_binned.groupby(['Age Group'])
# json_data_binned.describe()

In [10]:
player_count['Total Players'][0]

573

In [11]:
age_data['SN'].nunique()

Age Group
< 10        22
10 - 15     54
15 - 20    139
20 - 30    286
30 - 40     69
40 - 50      3
50 - 60      0
60 - 80      0
80+          0
Name: SN, dtype: int64

In [12]:
age_demographics = pd.concat([
    age_data['SN'].nunique(),
    round(100*(age_data['SN'].nunique()/player_count['Total Players'][0]),2)
    ], axis = 1)
age_demographics.columns=['Total Count','Percentage of Players']
age_demographics

,Total Count,Percentage of Players
Age Group,,
< 10,22,3.84
10 - 15,54,9.42
15 - 20,139,24.26
20 - 30,286,49.91
30 - 40,69,12.04
40 - 50,3,0.52
50 - 60,0,0.00
60 - 80,0,0.00
80+,0,0.00


# Purchasing Analysis (Age)

In [13]:
purchase_by_age = pd.concat([
    age_data['SN'].count(),
    round(age_data['Price'].mean(),2),
    round(age_data['Price'].sum(),2),
    round(age_data['Price'].sum()/age_data['SN'].nunique(),2)
    ], axis = 1)
purchase_by_age.columns = ['Purchase Count','Average Price','Total Value','Normalized Total']
# remove empty rows
purchase_by_age.dropna(how = 'any', inplace=True)
purchase_by_age

,Purchase Count,Average Price,Total Value,Normalized Total
Age Group,,,,
< 10,32,3.02,96.62,4.39
10 - 15,78,2.87,224.15,4.15
15 - 20,184,2.87,528.74,3.80
20 - 30,381,2.95,1122.43,3.92
30 - 40,102,3.00,305.75,4.43
40 - 50,3,2.88,8.64,2.88


# Top Spenders

In [14]:
# group by SN, sort by sum of purchase
individuals = json_data.groupby(['SN'])
purchase_by_name = pd.concat([
    individuals['Price'].sum(),
    individuals['Price'].count(),
    round(individuals['Price'].mean(),2)], axis = 1)
purchase_by_name.columns=['Total Purchase','Count of Purchases','Average Purchase']
purchase_by_name.sort_values(by = 'Total Purchase' ,ascending = False).head()

,Total Purchase,Count of Purchases,Average Purchase
SN,,,
Undirrala66,17.06,5,3.41
Saedue76,13.56,4,3.39
Mindimnya67,12.74,4,3.18
Haellysu29,12.73,3,4.24
Eoda93,11.58,3,3.86


# Most Popular Items

In [17]:
# group by item name, sort by purchase count
items = json_data.groupby(['Item ID'])
purchase_by_item = pd.concat([
    items['Item Name'].min(),
    items['Price'].count(),
    items['Price'].min(),
    items['Price'].sum()], axis = 1)
purchase_by_item.columns=['Item Name','Count of Purchases','Item Price','Total Purchase Value']
purchase_by_item.sort_values(by = 'Count of Purchases' ,ascending = False).head(10)

,Item Name,Count of Purchases,Item Price,Total Purchase Value
Item ID,,,,
39,"Betrayal, Whisper of Grieving Widows",11,2.35,25.85
84,Arcane Gem,11,2.23,24.53
31,Trickster,9,2.07,18.63
175,Woeful Adamantite Claymore,9,1.24,11.16
13,Serenity,9,1.49,13.41
34,Retribution Axe,9,4.14,37.26
65,Conqueror Adamantite Mace,8,1.96,15.68
152,Darkheart,8,3.15,25.20
44,Bonecarvin Battle Axe,8,2.46,19.68


# Most Profitable Items

In [16]:
purchase_by_item.sort_values(by = 'Total Purchase Value' ,ascending = False).head()

,Item Name,Count of Purchases,Item Price,Total Purchase Value
Item ID,,,,
34,Retribution Axe,9,4.14,37.26
115,Spectral Diamond Doomblade,7,4.25,29.75
32,Orenmir,6,4.95,29.70
103,Singed Scalpel,6,4.87,29.22
107,"Splitter, Foe Of Subtlety",8,3.61,28.88
